# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to("mps")

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Ensure tokenizer has padding token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

In [2]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
print (ds)
print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})
{'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?', 'role': 'user'}, {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suit

In [3]:
# TODO: 🦁 If your dataset is not in a 'chatml' format, you will need to process it. Refer to the [module](../chat_templates.md)

def process_dataset(sample):
    
    chat_messages = sample['messages']
    
    
    chat_formatted_sample = tokenizer.apply_chat_template(
        chat_messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
   
    sample['content']=chat_formatted_sample
    return sample
    

ds = ds.map(process_dataset)

for split in ds.keys():
    ds[split] = ds[split].remove_columns(['full_topic', 'messages'])
    

print (ds['train'][0])
print (ds)

{'content': "<|im_start|>user\nHi there<|im_end|>\n<|im_start|>assistant\nHello! How can I help you today?<|im_end|>\n<|im_start|>user\nI'm looking for a beach resort for my next vacation. Can you recommend some popular ones?<|im_end|>\n<|im_start|>assistant\nSome popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.<|im_end|>\n<|im_start|>user\nThat sounds great. Are there any resorts in the Caribbean that are good for families?<|im_end|>\n<|im_start|>assistant\nYes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.<|im_end|>\n<|im_start|>user\nOkay, I'll look into those. Thanks for the recommendations!<|im_end|>\n<|im_start|>assistant\nYou're welcome. I hope you find the perfect resort for your vacation.<|im_end|>\n<|im_start|>assistant\n"}
DatasetDict({
    train: D

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [4]:
# Configure the SFTTrainer

sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50  # Frequency of evaluation

    )    
# Prepare datasets
train_dataset = ds['train']
test_dataset = ds['test']


# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=test_dataset,
    dataset_text_field="content"
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/Users/alex/Projects/huggingface_smol-course/venv/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/alex/Projects/huggingface_smol-course/venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/alex/Projects/huggingface_smol-course/venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/Users/alex/Projects/huggingface_smol-course/venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:3

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [5]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./sft_output")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6798, 'grad_norm': 2.901040554046631, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2494, 'grad_norm': 2.694352865219116, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.2207, 'grad_norm': 2.367814064025879, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.1429, 'grad_norm': 2.453129768371582, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0467, 'grad_norm': 2.227726697921753, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1397372484207153, 'eval_runtime': 7.2733, 'eval_samples_per_second': 16.361, 'eval_steps_per_second': 2.062, 'epoch': 0.09}
{'loss': 1.1347, 'grad_norm': 2.275686502456665, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1442, 'grad_norm': 2.0798559188842773, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1034, 'grad_norm': 2.0705933570861816, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.0659, 'grad_norm': 2.234506130218506, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.0912, 'grad_norm': 2.038027048110962, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.0029182434082031, 'eval_runtime': 8.6574, 'eval_samples_per_second': 13.745, 'eval_steps_per_second': 1.733, 'epoch': 1.59}
{'loss': 0.8183, 'grad_norm': 1.674742579460144, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.792, 'grad_norm': 1.7766870260238647, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.7951, 'grad_norm': 2.054053544998169, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.778, 'grad_norm': 1.6900800466537476, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.8467, 'grad_norm': 1.9533995389938354, 'learning_rate': 2.5e-06, 'epoch': 1.68}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0019457340240479, 'eval_runtime': 7.2855, 'eval_samples_per_second': 16.334, 'eval_steps_per_second': 2.059, 'epoch': 1.68}
{'loss': 0.8446, 'grad_norm': 1.8085697889328003, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.768, 'grad_norm': 1.7071309089660645, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7798, 'grad_norm': 1.7438476085662842, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8203, 'grad_norm': 1.7668752670288086, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.7756, 'grad_norm': 1.7217768430709839, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0017584562301636, 'eval_runtime': 7.3378, 'eval_samples_per_second': 16.217, 'eval_steps_per_second': 2.044, 'epoch': 1.77}
{'train_runtime': 2389.0384, 'train_samples_per_second': 1.674, 'train_steps_per_second': 0.419, 'train_loss': 0.9431812148094177, 'epoch': 1.77}


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.